In [1]:
import sys 
sys.path.append("../")

In [2]:
import torch
import torch.nn as nn

In [82]:
def winit(m):
    classname = m.__class__.__name__
    if classname.find("Norm") != -1:
        nn.init.uniform_(m.weight)
        nn.init.zeros_(m.bias)
    elif classname.find("GRU") != -1:
        for w in m.all_weights:
            nn.init.zeros_(m[0].data)
            nn.init.zeros_(m[1].data)
            nn.init.uniform_(m[2].data)
            nn.init.zeros_(m[3].data)

In [86]:
gn = nn.Embedding(50, 20)
bn = nn.BatchNorm2d(50)
relu = nn.ReLU()
cn = nn.Conv2d(100, 50, 3)

# BatchNorm GroupNorm LayerNorm

In [87]:
gn.__class__.__name__

'Embedding'

In [94]:
gn.weight.data.size()

torch.Size([50, 20])

In [83]:
model = nn.Sequential(*[rn, gn, relu])

In [69]:
rn = nn.LSTM(100, 50, 3)

In [81]:
rn.all_weights[0][1].data

tensor([[ 0.0354, -0.0006,  0.0060,  ...,  0.0812,  0.1178,  0.0274],
        [ 0.0372,  0.0336,  0.0976,  ...,  0.1332,  0.0805,  0.1171],
        [ 0.0868,  0.1372,  0.0548,  ..., -0.0350,  0.1162, -0.1342],
        ...,
        [ 0.1218,  0.1092,  0.0858,  ...,  0.1023,  0.0617,  0.0242],
        [ 0.1101,  0.1058,  0.1374,  ...,  0.0470,  0.0639,  0.1042],
        [ 0.1027,  0.1238, -0.0198,  ..., -0.1163,  0.0175,  0.1261]])

In [84]:
model

Sequential(
  (0): LSTM(100, 50, num_layers=3)
  (1): Bilinear(in1_features=50, in2_features=50, out_features=20, bias=True)
  (2): ReLU()
)

In [85]:
model.apply(winit)

Sequential(
  (0): LSTM(100, 50, num_layers=3)
  (1): Bilinear(in1_features=50, in2_features=50, out_features=20, bias=True)
  (2): ReLU()
)